In [1]:
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime, timedelta
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

In [2]:
def pre_processing_current_data(new_data_path):
    # Define file paths
    log_term_client_data_path = 'D:\\Stock Recommendation data\\long_term_client_name.csv'
    
    # Load the data
    log_term_client = pd.read_csv(log_term_client_data_path)
    data = pd.read_csv(new_data_path)
    
    # Clean column names by stripping and replacing spaces
    data.columns = data.columns.str.strip().str.replace(' ', '')
    
    # Filter data based on long-term client names
    client_names = set(log_term_client['ClientName'])
    data = data[data['ClientName'].isin(client_names)].reset_index(drop=True)
    
    # Convert columns to appropriate types
    data['QuantityTraded'] = data['QuantityTraded'].str.replace(',', '').astype(int)
    data['TradePrice/Wght.Avg.Price'] = data['TradePrice/Wght.Avg.Price'].str.replace(',', '').astype(float)
    
    # Normalize 'Buy/Sell' values and adjust 'QuantityTraded'
    data['Buy/Sell'] = data['Buy/Sell'].str.upper().map({'BUY': 1, 'SELL': -1})
    data['QuantityTraded'] = data['QuantityTraded'].abs() * data['Buy/Sell']
    
    # Drop unnecessary columns
    data.drop(['Remarks'], axis=1, inplace=True)
    
    # Convert 'Date' to datetime
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Calculate 'Trade_values'
    data['Trade_values'] = data['QuantityTraded'] * data['TradePrice/Wght.Avg.Price']
    
    return data

In [3]:
def current_data_merge(current_data):
    # Path to the historical data CSV file
    historical_data_path = 'D:/Stock Recommendation data/Bulk_deal_data.csv'
    
    # Read the historical data
    historical_data = pd.read_csv(historical_data_path)
    
    # Convert 'Date' columns to datetime format
    historical_data['Date'] = pd.to_datetime(historical_data['Date'], format='%Y-%m-%d')
    current_data['Date'] = pd.to_datetime(current_data['Date'], format='%d-%m-%Y')
    
    # Check if there are any dates in current_data that are not in historical_data
    if not current_data['Date'].isin(historical_data['Date']).all():
        # Concatenate historical and current data
        main_data = pd.concat([historical_data, current_data], ignore_index=True)
        
        # Save the updated data back to the CSV file
        main_data.to_csv(historical_data_path, index=False)
    else:
        # If all dates in current_data are already in historical_data, return the historical data
        main_data = historical_data
    
    return main_data

In [4]:
def recommendations_system(data):
    today_date = datetime.today().date()
    previous_15 = today_date - timedelta(days=15)
    
    # Filter data for the last 30 days
    data['Date'] = pd.to_datetime(data['Date'])
    recent_data = data[data['Date'].dt.date > previous_15]

    # Group by 'SecurityName' and calculate total trade values
    position_data = recent_data.groupby('Symbol')['Trade_values'].sum().reset_index()
    
    Buy_record = []
    Sell_record = []

    for stock_name in position_data['Symbol'].unique():
        pos = recent_data[recent_data['Symbol'] == stock_name]
        total_value = round(pos['Trade_values'].sum(), 2)
        last_three_position = pos['Buy/Sell'].iloc[-3:].sum()

        if total_value > 0 and last_three_position >= 3:
            Buy_record.append({
                'Single': 'Buy',
                'Last Position Date': pos.iloc[-1]['Date'],
                'Stock Name': pos.iloc[-1]['Symbol'],
                'Last Position Client Name': pos.iloc[-1]['ClientName'],
                'Last Position Trade Price': pos.iloc[-1]['TradePrice/Wght.Avg.Price'],
                'Last Position Quantity': pos.iloc[-1]['QuantityTraded'],
                'Total Buy Trade value': total_value
            })
        elif total_value < 0 and last_three_position <= -3:
            Sell_record.append({
                'Single': 'Sell',
                'Last Position Date': pos.iloc[-1]['Date'],
                'Stock Name': pos.iloc[-1]['Symbol'],
                'Last Position Client Name': pos.iloc[-1]['ClientName'],
                'Last Position Trade Price': pos.iloc[-1]['TradePrice/Wght.Avg.Price'],
                'Last Position Quantity': pos.iloc[-1]['QuantityTraded'],
                'Total Sell Trade value': total_value
            })

    # Create DataFrames from records
    buy = pd.DataFrame(Buy_record)
    sell = pd.DataFrame(Sell_record)

    # Sort and filter records by date
    if not buy.empty:
        buy = buy.sort_values(by='Last Position Date').reset_index(drop=True)
        buy = buy[buy['Last Position Date'].dt.date > previous_15]

    if not sell.empty:
        sell = sell.sort_values(by='Last Position Date').reset_index(drop=True)
        sell = sell[sell['Last Position Date'].dt.date > previous_15]

    # Convert to list of dictionaries
    Buy_recommendations = buy.to_dict(orient='records') if not buy.empty else []
    Sell_recommendations = sell.to_dict(orient='records') if not sell.empty else []

    return Buy_recommendations, Sell_recommendations

In [5]:
if __name__ == '__main__':
    data_record_path = 'D:/Stock Recommendation data/Recommendation_stock_data.csv'
    new_data_path = 'D:\\Stock Recommendation data\\Bulk-Deals-04-06-2024-to-11-06-2024.csv'
    data = pre_processing_current_data(new_data_path = new_data_path)
    main_data = current_data_merge(current_data=data)
    Buy_recommendations, Sell_recommendations = recommendations_system(data=main_data)
    today_date_str = datetime.today().strftime("%d-%m-%Y")
    record = pd.read_csv(data_record_path)
    data_list = []
    print('************************BUY RECOMMENDATION SHARE****************************')
    for recommendation in Buy_recommendations:
        print('- Recommendation:', recommendation['Single'])
        print('  Last Position Date:', recommendation['Last Position Date'])
        print('  Stock Name:', recommendation['Stock Name'])
        print('  Last Position Client Name:', recommendation['Last Position Client Name'])
        print('  Last Position Trade Price:', recommendation['Last Position Trade Price'])
        print('  Last Position Quantity:', '{:,}'.format(recommendation['Last Position Quantity']))
        print('  Total Buy Trade value:', '{:,.2f}'.format(recommendation['Total Buy Trade value']))
        print()
        data_buy = {
            'Recommendation_Date' : today_date_str,
            'Recommendation' : recommendation['Single'],
            'Stock_Name' : recommendation['Stock Name'],
            'Client_Name' : recommendation['Last Position Client Name'],
            'Last_Position_Trade_Price' : recommendation['Last Position Trade Price'],
           'Last_Position_Quantity': '{:,}'.format(recommendation['Last Position Quantity']),
            'Total_Value' : '{:,.2f}'.format(recommendation['Total Buy Trade value']),
            'Last Position Date' : recommendation['Last Position Date'].date().strftime("%d-%m-%Y")
        }
        data_list.append(data_buy)
    print('************************SELL RECOMMENDATION SHARE****************************')
    for recommendation in Sell_recommendations:
        print('- Recommendation:', recommendation['Single'])
        print('  Last Position Date:', recommendation['Last Position Date'])
        print('  Stock Name:', recommendation['Stock Name'])
        print('  Last Position Client Name:', recommendation['Last Position Client Name'])
        print('  Last Position Trade Price:', recommendation['Last Position Trade Price'])
        print('  Last Position Quantity:', '{:,}'.format(recommendation['Last Position Quantity']))
        print('  Total Sell Trade value:', '{:,.2f}'.format(recommendation['Total Sell Trade value']))
        print()
        data_sell = {
            'Recommendation_Date' : today_date_str,
            'Recommendation' : recommendation['Single'],
            'Stock_Name' : recommendation['Stock Name'],
            'Client_Name' : recommendation['Last Position Client Name'],
            'Last_Position_Trade_Price' : recommendation['Last Position Trade Price'],
           'Last_Position_Quantity': '{:,}'.format(recommendation['Last Position Quantity']),
            'Total_Value' : '{:,.2f}'.format(recommendation['Total Sell Trade value']),
            'Last Position Date' : recommendation['Last Position Date'].date().strftime("%d-%m-%Y")
        }
        data_list.append(data_sell)
    data = pd.DataFrame(data_list)
    df = pd.concat([record , data] ,ignore_index=True)
    if not df['Recommendation_Date'].isin(record['Recommendation_Date']).all():
        df.to_csv(data_record_path , index= False)
    else:
        pass
    print('-----------------------------------------------------------------------------')


************************BUY RECOMMENDATION SHARE****************************
- Recommendation: Buy
  Last Position Date: 2024-06-06 00:00:00
  Stock Name: GSMFOILS
  Last Position Client Name: MULTIPLIER SHARE & STOCK ADVISORS PRIVATE LIMITED
  Last Position Trade Price: 33.4
  Last Position Quantity: 200,000
  Total Buy Trade value: 51,871,680.00

- Recommendation: Buy
  Last Position Date: 2024-06-06 00:00:00
  Stock Name: KSHITIJPOL
  Last Position Client Name: YMD FINANCIAL CONSULTANCY PRIVATE LIMITED
  Last Position Trade Price: 5.55
  Last Position Quantity: 396,780
  Total Buy Trade value: 7,497,550.72

- Recommendation: Buy
  Last Position Date: 2024-06-10 00:00:00
  Stock Name: CIGNITITEC
  Last Position Client Name: RAJASTHAN GLOBAL SECURITIES PVT LTD
  Last Position Trade Price: 1349.23
  Last Position Quantity: 185,675
  Total Buy Trade value: 756,306,305.85

************************SELL RECOMMENDATION SHARE****************************
- Recommendation: Sell
  Last Position